# Take home assignment - Front
## Ignacio Loyola Hernández

### Requirements

In [2]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

### Settings

In [12]:
API_BASE_URL = "https://api.weather.gov/stations"
STATION_ID = "0128W"  # Puedes cambiar este ID por el que prefieras

In [23]:
# get list of stations
def get_stations():
    response = requests.get(f"{API_BASE_URL}")
    stations = response.json().get('features', [])
    timezone_map = {station['properties']['stationIdentifier']: station['properties']['timeZone'] for station in stations}
    return timezone_map

In [25]:
time_zones = get_stations()
time_zones

{'0007W': 'America/New_York',
 '000PG': 'America/Los_Angeles',
 '000SE': 'America/Los_Angeles',
 '001HE': 'Pacific/Honolulu',
 '001HI': 'Pacific/Honolulu',
 '001MD': 'America/New_York',
 '001NH': 'America/New_York',
 '001PG': 'America/Los_Angeles',
 '001SE': 'America/Los_Angeles',
 '0021W': 'America/New_York',
 '002HE': 'Pacific/Honolulu',
 '002HI': 'Pacific/Honolulu',
 '002MD': 'America/New_York',
 '002NH': 'America/New_York',
 '002PG': 'America/Los_Angeles',
 '002SE': 'America/Los_Angeles',
 '003HE': 'Pacific/Honolulu',
 '003HI': 'Pacific/Honolulu',
 '003MD': 'America/New_York',
 '003NH': 'America/New_York',
 '003PG': 'America/Los_Angeles',
 '003SE': 'America/Los_Angeles',
 '0040W': 'America/New_York',
 '0043W': 'America/New_York',
 '0046W': 'America/New_York',
 '0047W': 'America/New_York',
 '0049W': 'America/New_York',
 '004HC': 'America/Chicago',
 '004HE': 'Pacific/Honolulu',
 '004HI': 'Pacific/Honolulu',
 '004MD': 'America/New_York',
 '004NH': 'America/New_York',
 '004PG': 'Americ

In [26]:
# get weather information for a single station
def get_station_observations(station_id):
    url = f"{API_BASE_URL}/{station_id}/observations"
    response = requests.get(url)
    observations = response.json().get('features', [])
    return observations

In [33]:
observations = get_station_observations(STATION_ID)
observations[0]['properties']

{'@id': 'https://api.weather.gov/stations/0128W/observations/2024-09-04T16:40:00+00:00',
 '@type': 'wx:ObservationStation',
 'elevation': {'unitCode': 'wmoUnit:m', 'value': 11},
 'station': 'https://api.weather.gov/stations/0128W',
 'timestamp': '2024-09-04T16:40:00+00:00',
 'rawMessage': '',
 'textDescription': '',
 'icon': None,
 'presentWeather': [],
 'temperature': {'unitCode': 'wmoUnit:degC',
  'value': 33.72,
  'qualityControl': 'V'},
 'dewpoint': {'unitCode': 'wmoUnit:degC',
  'value': 26.97,
  'qualityControl': 'V'},
 'windDirection': {'unitCode': 'wmoUnit:degree_(angle)',
  'value': 165,
  'qualityControl': 'V'},
 'windSpeed': {'unitCode': 'wmoUnit:km_h-1',
  'value': 1.62,
  'qualityControl': 'V'},
 'windGust': {'unitCode': 'wmoUnit:km_h-1',
  'value': None,
  'qualityControl': 'Z'},
 'barometricPressure': {'unitCode': 'wmoUnit:Pa',
  'value': 101693.2,
  'qualityControl': 'V'},
 'seaLevelPressure': {'unitCode': 'wmoUnit:Pa',
  'value': None,
  'qualityControl': 'Z'},
 'visib

In [30]:
# Transform and create data points
def transform_observations(observations, timezone_map):
    transformed_data = []
    timezone = timezone_map.get(STATION_ID, "Unknown")  # Get station's timezone
    for obs in observations:
        data = {
            "station_id": obs['properties']['station'],
            "station_name": obs['properties']['station'],
            "station_timezone": timezone,
            "latitude": obs['geometry']['coordinates'][1],
            "longitude": obs['geometry']['coordinates'][0],
            "timestamp": obs['properties']['timestamp'],
            "temperature": round(obs['properties']['temperature']['value'], 2) if obs['properties']['temperature'] else None,
            "wind_speed": round(obs['properties']['windSpeed']['value'], 2) if obs['properties']['windSpeed'] else None,
            "humidity": round(obs['properties']['relativeHumidity']['value'], 2) if obs['properties']['relativeHumidity'] else None,
        }
        transformed_data.append(data)
    return transformed_data

In [31]:
obs = transform_observations(observations, time_zones)

TypeError: type NoneType doesn't define __round__ method

In [11]:
transformed_data = transform_observations(observations)

KeyError: 'timeZone'